<font color='tomato'><font color="#CC3D3D"><p>
# DNN Modeling with ktrain

- ktrain은 tensorflow 기반의 low-code deep learning library임   
https://github.com/amaiya/ktrain  


- 아래 코드는 Seed를 설정하지 않았기 때문에 실행할 때마다 결과가 달라짐.    
여러번 실행해서 얻는 submission 화일을 앙상블해서 사용하는 것이 좋음.

### Imports

In [1]:
import ktrain
from ktrain import tabular  # tabular data에 적용
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

C:\Users\82105\anaconda3\envs\ktrain\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read Data

In [2]:
# 1st round 1등팀 피처 사용
X_train = pd.read_csv('seo_X_new_selected.csv', encoding='cp949')
X_test = pd.read_csv('seo_X_te_new_selected.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv').group
IDtest = pd.read_csv('X_test.csv', encoding= 'cp949').custid.unique()

In [3]:
# 아래 tabular_from_df()에서는 target 필드까지 들어간 데이터프레임을 요구
train_df = pd.concat([X_train, pd.DataFrame(y_train)], axis=1)

### Preprocessing

In [4]:
# Scaling, Imputation, train/valid 분할 등의 전처리를 수행하고 나중에 사용할 trn, val, preproc를 반환.
# validation data의 비율(현재 20%)은 val_pct로 지정
trn, val, preproc = tabular.tabular_from_df(train_df, label_columns=['group'], val_pct=0.2, random_state=42)

processing train: 17261 rows x 541 columns
['F20', 'F30', 'F40', 'F50', 'M20', 'M30', 'M40', 'M50']
   F20  F30  F40  F50  M20  M30  M40  M50
0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
processing test: 4326 rows x 541 columns
['F20', 'F30', 'F40', 'F50', 'M20', 'M30', 'M40', 'M50']
    F20  F30  F40  F50  M20  M30  M40  M50
1   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
7   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
11  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
12  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
33  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0


### Modeling

In [5]:
# MLP를 사용하여 모델링 (tabular data는 MLP만 지원). 배치 사이즈는 성능에 지대한 영향을 미침.
model = tabular.tabular_classifier('mlp', trn)
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=512)

Is Multi-Label? False
done.


In [6]:
# Model Fitting: 학습율(lr)과 에포크(epochs)가 성능에 지대한 영향을 미침.
hist = learner.fit_onecycle(lr=5e-3, epochs=4, checkpoint_folder='output')



begin training using onecycle policy with max lr of 0.005...
Epoch 1/4
34/34 [==============================] - 9s 276ms/step - loss: 1.7473 - accuracy: 0.3970 - val_loss: 1.5210 - val_accuracy: 0.4434
Epoch 2/4
34/34 [==============================] - 9s 273ms/step - loss: 1.5257 - accuracy: 0.4471 - val_loss: 1.5200 - val_accuracy: 0.4385
Epoch 3/4
34/34 [==============================] - 9s 277ms/step - loss: 1.4831 - accuracy: 0.4603 - val_loss: 1.5069 - val_accuracy: 0.4515
Epoch 4/4
34/34 [==============================] - 9s 276ms/step - loss: 1.3766 - accuracy: 0.4905 - val_loss: 1.4942 - val_accuracy: 0.4519


In [1]:
# Log Loss: 거의 대부분 val_loss 값이 낮을수록 LB 스코어도 낮음. 즉 val_loss의 변화가 학습에 절대적으로 중요!!!
plt.plot(hist.history["loss"], label="train")
plt.plot(hist.history["val_loss"], label="validation")
plt.legend()
plt.xlabel('epoch')
plt.title("Loss")
plt.show()

NameError: name 'plt' is not defined

### Prediction

In [8]:
predictor = ktrain.get_predictor(learner.model, preproc)
pred = predictor.predict(X_test, return_proba=True)

['F20', 'F30', 'F40', 'F50', 'M20', 'M30', 'M40', 'M50']
   F20  F30  F40  F50  M20  M30  M40  M50
0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
450/450 [==============================] - 26s 59ms/step


In [9]:
t = pd.Timestamp.now()
fname = f"ktrain_mlp_submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
pred = pd.DataFrame(pred)
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
submissions = pd.concat([pd.Series(IDtest, name="ID"), pred] ,axis=1)
submissions.to_csv(fname, index=False)
print("'{}' is ready to submit." .format(fname))

'ktrain_mlp_submission_06122010.csv' is ready to submit.


<font color='tomato'><font color="#CC3D3D"><p>
# End